<a href="https://colab.research.google.com/github/Lutris98/21_1Q/blob/master/1.Titanic/Titanic_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. EDA

In [101]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV #model_selection doesn't need mentioning, only the method does
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score #given metric was accuracy


In [102]:
from google.colab import drive #Korean Army banned me from using os resources so I learned new ways
drive.mount('/content/gdrive')
titanic_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/titanic_train.csv') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [103]:
titanic_df.head(3) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [104]:
print(titanic_df.info()) #objects are strings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [105]:
titanic_feature=titanic_df.drop(['Survived'], axis=1) #parameter axis is mandatory
titanic_label=titanic_df['Survived']

#2. Data Preprocessing

In [106]:
titanic_feature=titanic_feature.drop(['Name', 'Ticket'], axis=1) #1 #PassengerId will be useless but it is how the result should find people #drop method doesn't allow inplacing(Should check results along the way)
titanic_feature['Age'].fillna(titanic_feature['Age'].mean(), inplace=True) #could be affecting, so average would minimize the loss(for further development, something could be inferred from sibsp&parch)
titanic_feature['Cabin'].fillna('N', inplace=True) #inplacing so that additional object would be unnecesory
titanic_feature['Embarked'].fillna('N',inplace=True) #Wonder features like this would be better off set random...
print(titanic_feature.isnull().sum().sum()) #checking

0


In [107]:
print('Sex distribution:\n',titanic_feature['Sex'].value_counts()) #focusing on columns
print('Cabin distribution\n',titanic_feature['Cabin'].value_counts())
print('Embarked distribution:\n',titanic_feature['Embarked'].value_counts())

Sex distribution:
 male      577
female    314
Name: Sex, dtype: int64
Cabin distribution
 N              687
C23 C25 C27      4
B96 B98          4
G6               4
D                3
              ... 
C47              1
C118             1
B30              1
A26              1
C85              1
Name: Cabin, Length: 148, dtype: int64
Embarked distribution:
 S    644
C    168
Q     77
N      2
Name: Embarked, dtype: int64


In [108]:
titanic_feature['Cabin']=titanic_df['Cabin'].str[:1] #Cabin needs insight that only the first character contains information #pd.str is applicable to series
features=['Cabin','Sex','Embarked']
for feature in features:
  titanic_feature[feature]=LabelEncoder().fit_transform(titanic_feature[feature].astype(str)) #changing dtype of series

#3. Cross validation & Hyperparameter tuning

In [109]:
rf_clf=RandomForestClassifier(random_state=0) #Cross validation need models 
xgb_clf=XGBClassifier(random_state=0)
lgbm_clf=LGBMClassifier(random_state=0)
lr_clf=LogisticRegression(random_state=0, max_iter=5000)
X_train, X_test, y_train, y_test=train_test_split(titanic_feature, titanic_label, test_size=0.1,random_state=0)

In [110]:
rf_params={'n_estimators':[200,300],
           'max_depth':[6,8,10,12], #similar to decisiontree
           'min_samples_leaf':[3,5,7],
           'min_samples_split':[10,15,20]}
rf_gridcv=GridSearchCV(rf_clf, param_grid=rf_params, cv=5, n_jobs=-1) #using all CPU cores
rf_gridcv.fit(X_train, y_train)
print('best parameters:\n',rf_gridcv.best_params_)

best parameters:
 {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}


In [111]:
lgbm_params={'n_estimators':[200,300],
             'num_leaves':[32,64],
             'min_child_samples':[60,100],
             'subsample':[0.8,1],
             'max_depth':[10]}
lgbm_gridcv=GridSearchCV(lgbm_clf, param_grid=lgbm_params, cv=5, n_jobs=-1) 
lgbm_gridcv.fit(X_train, y_train)
print('best parameters:\n',lgbm_gridcv.best_params_)

best parameters:
 {'max_depth': 10, 'min_child_samples': 100, 'n_estimators': 200, 'num_leaves': 32, 'subsample': 0.8}


In [112]:
lr_params={'C':[0.1,1,10]}
lr_gridcv=GridSearchCV(lr_clf, param_grid=lr_params, cv=5, n_jobs=-1) 
lr_gridcv.fit(X_train, y_train)
print('best parameters:\n',lr_gridcv.best_params_)

best parameters:
 {'C': 0.1}


#4. modeling

In [113]:
rf_clf=RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=3, min_samples_split=10, random_state=0) #Cross validation need models 
xgb_clf=XGBClassifier(n_estimators=200, max_depth=10,learning_rate=0.1)
lgbm_clf=LGBMClassifier(n_estimators=300, num_leaves=32, min_child_samples=100, subsample=0.8, max_depth=10, random_state=0)
lr_clf=LogisticRegression(C=0.1)
vo_clf=VotingClassifier(estimators=[('RandomForest',rf_clf),('XGBoost',xgb_clf),('LightGBM',lgbm_clf),('LogisticRegression',lr_clf)], voting='soft')
vo_clf.fit(X_train, y_train)
pred1=rf_clf.fit(X_train, y_train).predict(X_test)
pred2=xgb_clf.fit(X_train, y_train).predict(X_test)
pred3=lgbm_clf.fit(X_train, y_train).predict(X_test)
pred4=lr_clf.fit(X_train, y_train).predict(X_test)
pred5=vo_clf.predict(X_test)
print('Accuracy1:{0:.4f} Accuracy2:{1:.4f} Accuracy3:{2:4f} Accuracy4:{3:4f}'.format(accuracy_score(y_test,pred1), accuracy_score(y_test,pred2),accuracy_score(y_test,pred3),accuracy_score(y_test,pred4)))
print('Final accuracy:{0:.4f}'.format(accuracy_score(y_test,pred5))) #from python3.6 should use fstring formatting  #:separates index and the number

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy1:0.8333 Accuracy2:0.8444 Accuracy3:0.855556 Accuracy4:0.766667
Final accuracy:0.8333


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# 1) Uploading

In [114]:
from google.colab import drive 
drive.mount('/content/gdrive')
test_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/titanic_test.csv') 
test_df.info()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [115]:
test_df=test_df.drop(['Name', 'Ticket'], axis=1) #uploading needs the same preprocessing I did on training set(but no need to split feature&label)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True) 
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True) 
test_df['Cabin'].fillna('N', inplace=True) 
test_df['Cabin']=test_df['Cabin'].str[:1] 
features=['Cabin','Sex','Embarked']
for feature in features:
  test_df[feature]=LabelEncoder().fit_transform(test_df[feature].astype(str)) 
realpred=vo_clf.predict(test_df) #output of classifier is ndarray object
realpred=pd.DataFrame(realpred, columns=['Survived']) #never forget[]
Submit=pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':realpred['Survived']}) #Data set by giving each column name and data
Submit.head(3)
Submit.to_csv('/content/gdrive/MyDrive/Dataset/Lutris_titanic.csv', index = False) #The client doesn't want indice on submission

# 2) Feedback<br>
